In [1]:
# https://www.kaggle.com/c/house-prices-advanced-regression-techniques

#RMSE <0.11 [Position under 100 out of 5,136 teams] 

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import ExtraTreesRegressor
from mlxtend.regressor import StackingCVRegressor

from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats

from sklearn.metrics import mean_squared_error
from math import sqrt

In [57]:

# Read train and test dataset
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Show all Columns in DF
pd.set_option('display.max_rows', train.shape[0])
pd.set_option('display.max_columns', train.shape[1])

In [58]:
train.shape

(1460, 81)

In [59]:
total = train.isnull().sum().sort_values(ascending=False)
#total[total > 0]

percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
#percent[percent > 0]

missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
#missing_data[missing_data > 0]
#missing_data[missing_data['Total'] > 1]

#Remove columns which have NUlls (dont agree with this step)
train = train.drop((missing_data[missing_data['Total'] > 1]).index,1)
#train.shape

#Remve the column in electrical that might be NULL
#train.loc[train['Electrical'].isnull()].index
train = train.drop(train.loc[train['Electrical'].isnull()].index)
#train.shape

train.isnull().sum().max() #just checking that there's no missing data missing.


0

In [60]:
# Test df missing data
total = test.isnull().sum().sort_values(ascending=False)
percent = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
test = test.drop((missing_data[missing_data['Total'] > 1]).index,1)
test = test.drop(test.loc[test['Electrical'].isnull()].index)
test.isnull().sum().max() #just checking that there's no missing data missing.

1

In [61]:
saleprice_scaled = StandardScaler().fit_transform(train['SalePrice'][:,np.newaxis]);
low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
high_range= saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]

In [62]:
train.shape

(1459, 63)

In [63]:
# Delete outliers points [Figure out using scatter plot which these outlier points are]
train.sort_values(by = 'GrLivArea', ascending = False)[:2]
train = train.drop(train[train['Id'] == 1299].index)
train = train.drop(train[train['Id'] == 524].index)
train.shape

(1457, 63)

In [64]:
# Logarithmic transformation in some variables
train['SalePrice'] = np.log(train['SalePrice'])
train['GrLivArea'] = np.log(train['GrLivArea'])
test['GrLivArea'] = np.log(test['GrLivArea'])
train.shape

(1457, 63)

In [65]:
#create column for new variable
train['HasBsmt'] = pd.Series(len(train['TotalBsmtSF']), index=train.index)
train['HasBsmt'] = 0 
train.loc[train['TotalBsmtSF'] > 0,'HasBsmt'] = 1

# transform data
train.loc[train['HasBsmt']==1,'TotalBsmtSF'] = np.log(train['TotalBsmtSF'])

#create column for new variable (one is enough because it's a binary categorical feature)
#if area>0 it gets 1, for area==0 it gets 0
test['HasBsmt'] = pd.Series(len(test['TotalBsmtSF']), index=test.index)
test['HasBsmt'] = 0 
test.loc[test['TotalBsmtSF']>0,'HasBsmt'] = 1

# transform data
test.loc[test['HasBsmt']==1,'TotalBsmtSF'] = np.log(test['TotalBsmtSF'])

train.shape

C:\Users\akapoor\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


(1457, 64)

In [66]:
test.shape

(1459, 58)

In [67]:
# Transform qualitative feature into quantitative feature
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# Separate price column from train table
y = train['SalePrice']
train = train.drop(["SalePrice"], axis = 1)
#train.shape

In [70]:
train.tail()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,HasBsmt,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,...,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1455,1456,60,7917,6,5,1999,2000,0,0,953,6.859615,953,694,0,7.406711,0,0,2,1,3,1,7,1,2,460,0,40,0,0,0,0,0,8,2007,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1456,1457,20,13175,6,6,1978,1988,790,163,589,7.340836,2073,0,0,7.636752,1,0,2,0,3,1,7,2,2,500,349,0,0,0,0,0,0,2,2010,1,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1457,1458,70,9042,7,9,1941,2006,275,0,877,7.049255,1188,1152,0,7.757906,0,0,2,0,4,1,9,2,1,252,0,60,0,0,0,0,2500,5,2010,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1458,1459,20,9717,5,6,1950,1996,49,1029,0,6.982863,1078,0,0,6.982863,1,0,1,0,2,1,5,0,1,240,366,0,112,0,0,0,0,4,2010,1,0,0,0,1,0,...,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1459,1460,20,9937,5,6,1965,1965,830,290,136,7.135687,1256,0,0,7.135687,1,0,1,1,3,1,6,0,1,276,736,68,0,0,0,0,0,6,2008,1,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [68]:
train.shape

(1457, 221)

In [69]:
test.shape

(1459, 192)

In [73]:
# Remove Missing Columns from train
missing_cols = set( train.columns ) - set( test.columns )
train.drop( missing_cols , inplace = True, axis = 1 )
#train.shape

# Features with no value is filled with the mean
mean = train.mean().astype(np.int32)
train.fillna( mean , inplace = True)
mean = test.mean().astype(np.int32)
test.fillna( mean , inplace = True)


In [74]:
# Check if dataframe has NULL elements
test.isnull().sum()
train.isnull().sum()

# See correlation between features and SalePrice
train.corrwith(y).sort_values(ascending=False)

OverallQual              0.821589
GrLivArea                0.737430
GarageCars               0.681053
GarageArea               0.656157
1stFlrSF                 0.620761
FullBath                 0.596021
YearBuilt                0.587301
YearRemodAdd             0.566208
TotRmsAbvGrd             0.537716
Foundation_PConc         0.531414
ExterQual_Gd             0.510033
Fireplaces               0.492159
HeatingQC_Ex             0.467098
KitchenQual_Ex           0.417482
KitchenQual_Gd           0.407084
BsmtFinSF1               0.392429
TotalBsmtSF              0.373552
ExterQual_Ex             0.362392
Neighborhood_NridgHt     0.351858
CentralAir_Y             0.351605
Exterior2nd_VinylSd      0.337860
Exterior1st_VinylSd      0.336551
WoodDeckSF               0.334250
SaleType_New             0.331362
SaleCondition_Partial    0.326547
OpenPorchSF              0.325277
2ndFlrSF                 0.319998
HalfBath                 0.314339
Electrical_SBrkr         0.305995
PavedDrive_Y  

In [18]:
# Create new features based on existent features

train['OverallRate'] = train['OverallQual'] / train['OverallCond']
train['OverallRate2'] = train['OverallQual'] + train['OverallCond']

train['AreaLivGarage'] = train['GrLivArea'] / train['GarageArea']
train['AreaLivGarage'] = train['GrLivArea'] + train['GarageArea']

train['areaPerCar'] = train['GarageArea'] / train['GarageCars']

# Create new features based on existent features

test['OverallRate'] = test['OverallQual'] / test['OverallCond']
test['OverallRate2'] = test['OverallQual'] + test['OverallCond']

test['AreaLivGarage'] = test['GrLivArea'] / test['GarageArea']
test['AreaLivGarage'] = test['GrLivArea'] + test['GarageArea']

test['areaPerCar'] = test['GarageArea'] / test['GarageCars']

# Fill Nan values with 0
train.fillna( 0 , inplace = True)
test.fillna( 0 , inplace = True)

In [76]:
test.shape

(1459, 192)

In [77]:
# Initialize KMeans with 800 Clusters, 300 maximum iterations, 0.0001 of tolerance
cluster = KMeans(n_clusters= 800, max_iter=300, tol=0.0001, verbose=0, random_state = 0, n_jobs=-1)

# Execute Kmeans in train dataframe using fit()
kmeans_train = cluster.fit(train)

# Get Labels to kmeans
labels_train = kmeans_train.labels_

# Predict cluster index for each sample
kmeans_testing = cluster.predict(test)

# Insert values from kmeans to train and test dataframe
train['cluster'] = labels_train
test['cluster'] = kmeans_testing

print("Data points = %i " % train.shape[0])
print("Features = %i " % train.shape[1])

Data points = 1457 
Features = 193 


In [78]:
train.tail()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,HasBsmt,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,...,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,cluster
1455,1456,60,7917,6,5,1999,2000,0,0,953,6.859615,953,694,0,7.406711,2,1,3,1,7,1,2,460,0,40,0,0,0,0,0,8,2007,1,0,1,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,276
1456,1457,20,13175,6,6,1978,1988,790,163,589,7.340836,2073,0,0,7.636752,2,0,3,1,7,2,2,500,349,0,0,0,0,0,0,2,2010,1,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,300
1457,1458,70,9042,7,9,1941,2006,275,0,877,7.049255,1188,1152,0,7.757906,2,0,4,1,9,2,1,252,0,60,0,0,0,0,2500,5,2010,1,0,1,0,0,0,1,0,...,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,69
1458,1459,20,9717,5,6,1950,1996,49,1029,0,6.982863,1078,0,0,6.982863,1,0,2,1,5,0,1,240,366,0,112,0,0,0,0,4,2010,1,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,525
1459,1460,20,9937,5,6,1965,1965,830,290,136,7.135687,1256,0,0,7.135687,1,1,3,1,6,0,1,276,736,68,0,0,0,0,0,6,2008,1,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,550


In [79]:
# Drop ID columns from dataframes
id = test['Id']
test.drop(['Id'], axis = 1, inplace = True)
train.drop(['Id'], axis = 1, inplace = True)

In [81]:
test.shape

(1459, 192)

In [82]:
# 0.05% of data will be used to test
x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.25, random_state=42 )

In [83]:
# Fit the model
et  = ExtraTreesRegressor(n_estimators=950 ,  max_features = 'auto', max_leaf_nodes=None, n_jobs= -1, random_state = 0, verbose = 0)
gbr = GradientBoostingRegressor()
lasso = Lasso()
xgbr = XGBRegressor() 
svr = SVR(kernel= 'rbf', gamma= 'auto', tol=0.001, C=100.0, max_iter=-1)
rf = RandomForestRegressor(n_estimators=900,  random_state=0)
lr = LinearRegression(fit_intercept=True, normalize=True, copy_X=True, n_jobs=-1)
knnR = KNeighborsRegressor(n_neighbors=20, n_jobs=-1)
reg = StackingCVRegressor(regressors=[xgbr , gbr, lr, et], meta_regressor=lr)

reg.fit(x_train.values, y_train.values)

StackingCVRegressor(cv=5,
                    meta_regressor=LinearRegression(copy_X=True,
                                                    fit_intercept=True,
                                                    n_jobs=-1, normalize=True),
                    n_jobs=None, pre_dispatch='2*n_jobs', random_state=None,
                    refit=True,
                    regressors=[XGBRegressor(base_score=None, booster=None,
                                             colsample_bylevel=None,
                                             colsample_bynode=None,
                                             colsample_bytree=None, gamma=None,
                                             gpu_id=None,
                                             importance_type='gain',
                                             inte...
                                                    max_depth=None,
                                                    max_features='auto',
                                  

In [84]:
print('Test')
print(reg.score(x_test.values, y_test.values))
print('\nRMSLE')
print(sqrt(mean_squared_error(reg.predict(x_test.values), y_test)))
print('\nTrain')
print(reg.score(x_train.values, y_train.values))


Test
0.9066688713802983

RMSLE
0.12500821756717959

Train
0.9911729917464054


In [ ]:
# https://github.com/viniciosbarretos/Kaggle-House-Prices-Predict/blob/master/house-prices.ipynb

#  viniciosbarretos/Kaggle-House-Prices-Predict https://www.kaggle.com/c/house-prices-advanced-regression-techniques